## 扱う質問

自然科学に関する一般的な質問を取り扱います。特に質問<SUB>5<SUP>はRAGなしでは答えらないものになっています。

- 「ヒトの細胞内でATPを生成する主な経路は何ですか？」
-「2008年にノーベル物理学賞を受賞した日本人は誰ですか？」
-「エッシャーの作品に見られる特徴的な幾何学構造とは何か？」
-「日本における高齢化率は2020年時点で何％だったか？」
-「2024年3月に京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題となった技術は何か？」

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。



### 演習環境の準備

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52 (from 1)
Receiving objects: 100% (52/52), 83.21 KiB | 747.00 KiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving knowledge.txt to knowledge.txt


# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [ ]:
def generate_output(query):
  messages = [
      {"role": "user", "content": query},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
questions = [
    "ヒトの細胞内でATPを生成する主な経路は何ですか？",
    "2008年にノーベル物理学賞を受賞した日本人は誰ですか？",
    "2024年3月に京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題となった技術は何か？"
]

for question in questions:
    response = generate_output(question)
    print(f"Q: {question}")
    print(f"A: {response}")
    print("-" * 50)


Q: ヒトの細胞内でATPを生成する主な経路は何ですか？
A: ヒトの細胞内でATPを生成する主な経路は**細胞呼吸**です。 

具体的には、以下の3つの段階でATPを生成します。

1. **糖質分解:**  細胞内で摂取した糖質（主に glucose）が、細胞呼吸の最初の段階で分解されます。
2. ** Krebs cycle (ケロブサイクル):**  分解された糖質からエネルギーを抽出し、エネルギーを蓄えるためのエネルギー通貨であるATPを生成します。
3. **電子伝達系:**  ケロブサイクルで生成されたエネルギーを、最終的に電子伝達系でエネルギーを細胞に供給します。


 



--------------------------------------------------
Q: 2008年にノーベル物理学賞を受賞した日本人は誰ですか？
A: 2008年にノーベル物理学賞を受賞した日本人は **高田秀夫** です。 




--------------------------------------------------
Q: 2024年3月に京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題となった技術は何か？
A:  2024年3月に京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題になった技術は多岐に渡ります。 

いくつか例を挙げますね：

**1. iPS細胞による疾患モデルの開発:**

* **高精度な疾患モデルの構築:**  iPS細胞を用いた疾患モデルの構築がより高度になり、よりリアルな病態を再現できるようになりました。
* **個別化医療への応用:**  患者自身のiPS細胞を用いて、より正確な治療法の開発や、患者に合わせた治療計画の提案が可能になりました。

**2. AIによるiPS細胞の解析・診断:**

* **iPS細胞の特性解析:**  AIがiPS細胞の遺伝子発現や細胞機能を解析し、疾患のメカニズムをより深く理解するのに役立ちました。
* **iPS細胞による治療効果予測:**  AIがiPS細胞を用いた治療効果を予測する技術が開発され、治療の効率性向上に貢献しています。
* **iPS細胞による診断:**  AIがiPS細胞から得られた情報に基づいて、疾患の診断や病態の評価

### 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「ヒトの細胞内でATPを生成する主な経路は何ですか？」について一般的でない知識を提示しました：


# 2. 文字起こしデータの活用

## 2.1 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: LLM講座第4講における講師の発言内容
* **目的**: モデルに「Inference Time Scaling」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 音声認識モデル(speech2text)で書き起こした生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
with open("/content/knowledge.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[4])

ドキュメントサイズ:  11
ドキュメントの例: 
 M.C.エッシャーの作品には、視覚的パラドックスや非ユークリッド幾何学を応用した構造が多く用いられている


In [ ]:
# Retrievalの実行
question = "ヒトの細胞内でATPを生成する主な経路は何ですか？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[83.85194396972656, 74.57930755615234, 62.10759735107422, 62.74241256713867, 60.0194206237793, 62.91181945800781, 60.017574310302734, 60.99265670776367, 63.89998245239258, 61.34072494506836, 56.49089813232422]]


In [ ]:
questions = [
    "ヒトの細胞内でATPを生成する主な経路は何ですか？",
    "2008年にノーベル物理学賞を受賞した日本人は誰ですか？",
    "2024年3月に京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題となった技術は何か？"
]

topk = 3

for q in questions:
    print(f"[質問] {q}\n")

    # クエリ埋め込み生成
    query_embeddings = emb_model.encode([q], prompt_name="query")
    document_embeddings = emb_model.encode(documents)

    # 類似度スコア計算
    scores = (query_embeddings @ document_embeddings.T) * 100

    # 上位topk文書を表示
    for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
        print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
        print(documents[index], "\n")

    # 参考文書をまとめてプロンプト作成
    references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
    query = f"[参考資料]\n{references}\n\n[質問] {q}"

    # 回答生成
    response = generate_output(query)
    print(f"[回答]\n{response}")
    print("=" * 80)


[質問] ヒトの細胞内でATPを生成する主な経路は何ですか？

取得したドキュメント1: (Score: 83.85194396972656)
ヒトの細胞は、主にミトコンドリア内で行われる好気呼吸によってATPを産生している 

取得したドキュメント2: (Score: 74.57930755615234)
好気呼吸は解糖系、クエン酸回路、電子伝達系の3段階からなり、最終的に酸素を使って大量のATPを生成する 

取得したドキュメント3: (Score: 63.89998245239258)
2024年3月に京都大学で開催された医療AIカンファレンスでは、iPS細胞由来細胞の分化段階を画像から予測する深層学習モデルに注目が集まった 

[回答]
ヒトの細胞内でATPを生成する主な経路は**好気呼吸**です。


 



[質問] 2008年にノーベル物理学賞を受賞した日本人は誰ですか？

取得したドキュメント1: (Score: 88.31318664550781)
2008年のノーベル物理学賞は、小林誠氏と益川敏英氏、およびアメリカの南部陽一郎氏に授与された 

取得したドキュメント2: (Score: 72.10948181152344)
彼らはクォークの世代構造に関する理論で受賞した 

取得したドキュメント3: (Score: 67.85725402832031)
2024年3月に京都大学で開催された医療AIカンファレンスでは、iPS細胞由来細胞の分化段階を画像から予測する深層学習モデルに注目が集まった 

[回答]
2008年にノーベル物理学賞を受賞した日本人は、小林誠氏、益川敏英氏、およびアメリカの南部陽一郎氏です。 




[質問] 2024年3月に京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題となった技術は何か？

取得したドキュメント1: (Score: 82.70611572265625)
2024年3月に京都大学で開催された医療AIカンファレンスでは、iPS細胞由来細胞の分化段階を画像から予測する深層学習モデルに注目が集まった 

取得したドキュメント2: (Score: 62.37698745727539)
2008年のノーベル物理学賞は、小林誠氏と益川敏英氏、およびアメリカの南部陽一郎氏に授与された 

取得したドキ

### 結果

講義内容のドキュメントを追加したにもかかわらず、モデルの回答には依然として以下の問題が見られます：
* 「高速に推論する」など、従来の一般的な推論最適化と「Inference Time Scaling」を混同した誤った解釈が継続
* 講義内容を参照しているものの、概念の本質を正確に捉えられていない

###改善点
* aa
* bb

### 問題分析
以下の要因が考えられます：
1. **ドキュメント品質の問題**: 音声認識による文字起こしの精度不足
2. **検索精度の課題**: 単純な文単位の分割では文脈が失われ、関連性の高いドキュメント片を適切に取得できていない可能性

# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [ ]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] ヒトの細胞内でATPを生成する主な経路は何ですか？"
response = generate_output(query)
print(response)

ヒトの細胞内でATPを生成する主な経路は**好気呼吸**です。


 





In [ ]:
questions = [
    "ヒトの細胞内でATPを生成する主な経路は何ですか？",
    "2008年にノーベル物理学賞を受賞した日本人は誰ですか？",
    "2024年3月に京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題となった技術は何か？"
]

topk = 3

for q in questions:
    print(f"[質問] {q}\n")

    # クエリの埋め込み
    query_embeddings = emb_model.encode([q], prompt_name="query")
    document_embeddings = emb_model.encode(documents)

    # 類似度スコア計算
    scores = (query_embeddings @ document_embeddings.T) * 100

    # 前後2文ずつまとめて5文のチャンクを作成
    retrieved_chunks = []
    for i in scores.argsort()[0][::-1][:topk]:
        start = max(0, i - 2)
        end = min(i + 3, len(documents))  # +3 because slicing is exclusive
        chunk = "。".join(documents[start:end]).strip()
        retrieved_chunks.append("* " + chunk)

    references = "\n".join(retrieved_chunks)

    # 質問と参照資料を結合
    query = f"[参考資料]\n{references}\n\n[質問] {q}"

    # 回答生成
    response = generate_output(query)
    print(f"[回答]\n{response}")
    print("=" * 80)


[質問] ヒトの細胞内でATPを生成する主な経路は何ですか？

[回答]
ヒトの細胞内でATPを生成する主な経路は**好気呼吸**です。


 



[質問] 2008年にノーベル物理学賞を受賞した日本人は誰ですか？

[回答]
2008年にノーベル物理学賞を受賞した日本人は、小林誠氏、益川敏英氏、アメリカの南部陽一郎氏です。 




[質問] 2024年3月に京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題となった技術は何か？

[回答]
京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題となった技術は、**iPS細胞由来細胞の分化段階を画像から予測する深層学習モデル**です。 


 





## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られました：

### 改善点
* 「推論時の計算をスケールさせる」という概念を据えて回答
* Inference Time Scalingの基本原理についての理解が向上

### 残存する問題点
* 質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇




# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『LLMにおけるInference Time Scalingとは？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、Inference Time Scalingに関する正確で一貫性のある回答生成が期待されます。

In [ ]:
questions = [
    "ヒトの細胞内でATPを生成する主な経路は何ですか？",
    "2008年にノーベル物理学賞を受賞した日本人は誰ですか？",
    "2024年3月に京都大学で開催されたiPS細胞に関する医療AIカンファレンスで話題となった技術は何か？"
]

topk = 3

for question in questions:
    print(f"\n{'='*80}\n[質問] {question}\n")

    # クエリの埋め込みとスコア計算
    query_embeddings = emb_model.encode([question], prompt_name="query")
    document_embeddings = emb_model.encode(documents)
    scores = (query_embeddings @ document_embeddings.T) * 100

    # 関連文フィルタリング
    references = []
    candidate_refs = [
        "。".join(documents[max(0, i-2): min(i+3, len(documents))]).strip()
        for i in scores.argsort()[0][::-1][:topk]
    ]

    for ref in candidate_refs:
        relevance_query = (
            f"与えられた[参考資料]が[質問]に直接関連しているかを、'yes''no'で答えること。\n"
            f"[参考資料]\n{ref}\n\n[質問] {question}"
        )
        response = generate_output(relevance_query)

        print("▼ 対象ドキュメント:\n", ref.replace("。", "。\n"))
        print("→ 関連しているか: ", response)

        if "yes" in response.lower():
            references.append(ref)

    # 最終プロンプトと回答生成
    if references:
        joined_refs = "\n".join(["* " + r for r in references])
        query = f"[参考資料]\n{joined_refs}\n\n[質問] {question}"
    else:
        query = f"[質問] {question}（参考資料なし）"

    final_response = generate_output(query)
    print(f"\n[最終回答]\n{final_response}")



[質問] ヒトの細胞内でATPを生成する主な経路は何ですか？

▼ 対象ドキュメント:
 ヒトの細胞は、主にミトコンドリア内で行われる好気呼吸によってATPを産生している。
好気呼吸は解糖系、クエン酸回路、電子伝達系の3段階からなり、最終的に酸素を使って大量のATPを生成する。
2008年のノーベル物理学賞は、小林誠氏と益川敏英氏、およびアメリカの南部陽一郎氏に授与された
→ 関連しているか:  yes 



▼ 対象ドキュメント:
 ヒトの細胞は、主にミトコンドリア内で行われる好気呼吸によってATPを産生している。
好気呼吸は解糖系、クエン酸回路、電子伝達系の3段階からなり、最終的に酸素を使って大量のATPを生成する。
2008年のノーベル物理学賞は、小林誠氏と益川敏英氏、およびアメリカの南部陽一郎氏に授与された。
彼らはクォークの世代構造に関する理論で受賞した
→ 関連しているか:  yes 



▼ 対象ドキュメント:
 総務省統計局のデータによると、2020年時点で日本の高齢化率（65歳以上人口の割合）は28.7％であった。
これは世界で最も高い水準の一つである。
2024年3月に京都大学で開催された医療AIカンファレンスでは、iPS細胞由来細胞の分化段階を画像から予測する深層学習モデルに注目が集まった。
特にMaxViTやViTを応用した画像回帰モデルが、MAE=10.9、R²=0.44の精度で分化予測を達成したことが報告された。

→ 関連しているか:  no 




[最終回答]
ヒトの細胞内でATPを生成する主な経路は**好気呼吸**です。


 




[質問] 2008年にノーベル物理学賞を受賞した日本人は誰ですか？

▼ 対象ドキュメント:
 ヒトの細胞は、主にミトコンドリア内で行われる好気呼吸によってATPを産生している。
好気呼吸は解糖系、クエン酸回路、電子伝達系の3段階からなり、最終的に酸素を使って大量のATPを生成する。
2008年のノーベル物理学賞は、小林誠氏と益川敏英氏、およびアメリカの南部陽一郎氏に授与された。
彼らはクォークの世代構造に関する理論で受賞した。
M.C.エッシャーの作品には、視覚的パラドックスや非ユークリッド幾何学を応用した構造が多く用いられている
→ 関連しているか:  yes 



▼ 対象ドキュメ

## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

### 達成された成果
* Inference Time Scalingに関する正確な情報を含んだ回答の生成
* 無関係な情報やノイズの排除
* 講義内容を反映した説明の実現 🎉

この結果から、RAGパイプラインにおける情報の質と関連性の重要性であり、検索で取得した情報を単に増やすだけでなく、その情報の関連性を精査する方法を学ぶことができました。